<a href="https://colab.research.google.com/github/anxosanchez/sopq/blob/main/C%C3%A1lculos_PVT_para_gases_non_ideais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cálculos PVT para gases reais

Anxo Sánchez

Escola de Enxeñaría Industrial
Grao en Química Industrial

## Enunciado do problema

Os cartuchos comerciais de CO<sub>2</sub> teñen un gran número de usos, incluíndo inflado de reumáticos de bicicleta e chalecos salvavidas, dispensadores de refrescos, limpadores de gas comprimido para dispositivos electrónicos e pistolas de gas. Pódense comprar a baixo custo nunha tenda de deportes i en lineais. Un tamaño de cartucho de CO<sub>2</sub> popular ten capacidade para 12 g de gas, contido nun volume interno de 17,6 cm<sup>3</sup>. Estimar a presión dentro do cartucho nas condicións de 40 ºC polos seguintes métodos¨: E

1. Ecuación dos gases ideais.
2. Ecuación de __Van der Waals__.
2. Diagramas de compresibilidade (https://pubs.acs.org/doi/pdf/10.1021/ie50467a036).
3. Modelo do __Virial__. 
4. Ecuación de estado de __Soave-Redlich-Kwong__. 
5. Datos de referencia do __NIST Webbook__.

## Cuestións
Comparar os resultados obtidos e os erros e responder ás seguintes cuestións:

1. Cómo depende a presión de CO<sub>2</sub> da temperatura no rango de 30 ºC a 50 ºC?.
2. Porque tanto erro na lei dos gases ideais?.
3. Porque os diagramas de compresibilidade funcionan tan ben neste caso?.
4. Débese esperar que funcionen tan ben en outros?.
5. Como cambiarían os cálculos se se coñecen a presión e a temperatura se debe calcular o volume molar?.

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# Datos do problema
M_CO2 = 44.009       # mol-1
n_CO2 = 12 / M_CO2   # gmol
R  = 0.08206         # atm-L/mol/K
temperatura  = 40 + 273.15     # K
Volume  = 0.0176          # L

# Datos atopados para o CO2
presion_critica     = 72.83494 # atm
temperatura_critica = 304.1282 # K
factor_acentrico     = 0.225    # Factor acéntirco de Pitzer ($\omega)

# Diccioario de datos para almacenar resultados
resultados = {}

## Gases ideais

A ecuación dos gases ideais é a seguinte:
    
$$
pV = nRT
$$

onde $p$ é a presión do gas, $V$ o volume, $n$ o número de moles, $R$ a __Consante dos gases__ e $T$ a temperatura. Cando $n$ = 1, $V$ corresponde ó volume molar. Despexando $p$:
  
$$
p = \frac {RT} {V}
$$
  

In [3]:
p_gas_ideal = n_CO2 * R * temperatura / Volume
volume_especifico = Volume / n_CO2

print('Presión do gas ideal    =', p_gas_ideal, 'atm', p_gas_ideal * 0.101325, 'Pa', 14.696 * p_gas_ideal, 'psia')
print('Volume específico molar = ', volume_especifico, 'L/gmol')
print('Densidade Molar         =', 1 / volume_especifico, 'gmol/L')
# Se quero formatealo un pouco mais "mono"
print('Presión usando o modelo de gas ideal = {:.2f} atm ou {:.2f} Pa'.format(p_gas_ideal, p_gas_ideal* 0.101325) )
print('Volume específico molar              = {:.4f} L/gmol ou {:.4f} ft3/mol'.format(volume_especifico, volume_especifico * 454 / 28.3) )
print('Densidade molar                      = {:.4f} L/gmol ou {:.4f} ft3/mol'.format(1 / volume_especifico, 1 / volume_especifico / 454 * 28.3) )

resultados['Gas ideal'] = p_gas_ideal

Presión do gas ideal    = 398.11726010588734 atm 40.33923138022904 Pa 5850.73125451612 psia
Volume específico molar =  0.06454653333333335 L/gmol
Densidade Molar         = 15.492698807475326 gmol/L
Presión usando o modelo de gas ideal = 398.12 atm ou 40.34 Pa
Volume específico molar              = 0.0645 L/gmol ou 1.0355 ft3/mol
Densidade molar                      = 15.4927 L/gmol ou 0.9657 ft3/mol


## Factor de compresibilidade

A carta de compresibilidade xeneralizada é unha ferramenta útil para predicir as propiedades dos gases ou mesturas de gas con precisión aceptable para a maioría dos casos da enxeñería química. Non obstante, o gráfico é incómodo de usar cando o volume é unha das propiedades coñecidas e non se sabe a presión nin a temperatura, porque a resposta debe ser atopada por proba e erro. Para remediar esta condición, o volume crítico ideal permite incorporar liñas de volume reducido ideais como parte do gráfico de compresibilidade. O volume crítico ideal defínese:
  
$$
V_{ci}=\frac{R T_{c}}{p_{c}}
$$
  
entón, o volume reducido ideal é:
  
$$
v_{r^{\prime}}=\frac{v_{\mathrm{real}}}{v_{ci}}
$$
  
logo:
  
$$
V_{r^{\prime}}=\frac{\frac{z R T}{p}}{\frac{R T_{c}}{p_{c}}}=Z \frac{T_{r}}{p_{r}}
$$
  
onde $T$ e $p$ defínense do xeito habitual:
  
$$
T_{r}=\frac{T}{T_{c}} \quad p_{r}=\frac{p}{p_{c}}
$$

### [Carta ou diagrama de Compresiblidade](http://eon.sdsu.edu/testhome/Test/solve/basics/tables/tablesRG/zNO.html)

A ompresibilidade defínese como:

\begin{equation}
z = \frac{P\hat{V}}{RT}
\end{equation}

o que, por definición, ten nun valor de $z=1$ para o gasideal. Os gases reais amosan unha significativa desviación de 1, xenerlmente incrementándose esta a medida que aumentan a presión e a temperatura do gas, ou seña, que as moñéculas do mesmo se xuntan cada vez mais, e debece ca temperatura.

\begin{align*}
T_r & = \frac{T}{T_c}\\
P_r & = \frac{P}{P_c}
\end{align*}

A [Carta de compresibilidade](http://eon.sdsu.edu/testhome/Test/solve/basics/tables/tablesRG/zNO.html) representa o factor de ocmpresibilidade medido para un número de especies químicas. Dados os valores da presión reducida e a temperatura reducidae, $T_r$ and $p_r$, se pode localizar o valor de $z$, que logo se pode usar para calcular o valor do volume molar $\hat{V}$.

![Carta de compresibilidade xeralizada](http://eon.sdsu.edu/testhome/Test/solve/basics/tables/tablesRG/zNO7.png)

Para o caso de que $\hat{V}$ é unha das variables coñecidas é convinte definir un **volume reducido ideal**. Primeiro hai que definir un volume crítico ideal:

\begin{equation}
\hat{V}^{ideal}_c = \frac{RT_c}{P_c}
\end{equation}

A razón do subíndice 'ideal' é porque é ficticio (os gases reais non son ideais).

Entón, o **volume reducido ideal** $\hat{V}^{ideal}_r$ é: 

\begin{equation}
\hat{V}^{ideal}_r = \frac{\hat{V}}{\hat{V}_c} = \frac{P_c\hat{V}}{RT_c}
\end{equation}

A representación da compresibildade se reforza con curvas de $\hat{V}^{ideal}_r$ constante. Dados os valores de $\hat{V}^{ideal}_r$ e ben $P_r$ ou $T_r$, se localiza un valor de $z$. A definición de compresibilidade se usa, entón, para resolver a ecuación de unha incógnita.

Polo tanto, o **volume reducido ideal** $\hat{V}^{ideal}_r$ é:

\begin{equation}
\hat{V}^{ideal}_r = \frac{\hat{V}}{\hat{V}_c} = \frac{P_c\hat{V}}{RT_c}
\end{equation}

In [4]:
temperatura_reducida = temperatura / temperatura_critica
volume_reducido_ideal = presion_critica * volume_especifico / (R * temperatura_critica )

print('Tempertura reducida =', temperatura_reducida)
print('Volume reducido ideal               =', volume_reducido_ideal)
# ou
print('Tempertura reducida   = {:.4f}'.format(temperatura_reducida) )
print('Volume reducido ideal = {:.4f}'.format(volume_reducido_ideal) )

# Buscando valores en (https://pubs.acs.org/doi/pdf/10.1021/ie50467a036)
z = 0.28

# Cálculo da presión

presion_compresibilidade = z * R * temperatura / volume_especifico

resultados['Compresibilidade'] = presion_compresibilidade

print('Presión de CO2 =', round(presion_compresibilidade, 3), 'atm', 14.696 * presion_compresibilidade, 'psia')
print('Presión calculada por carta de compresibilidade {:.4f} atm ou {:.4f} psia'.format(presion_compresibilidade,14.696 * presion_compresibilidade))

Tempertura reducida = 1.02966446386754
Volume reducido ideal               = 0.18837552892325715
Tempertura reducida   = 1.0297
Volume reducido ideal = 0.1884
Presión de CO2 = 111.473 atm 1638.2047512645136 psia
Presión calculada por carta de compresibilidade 111.4728 atm ou 1638.2048 psia


## Modelo do Virial 

A serie de expansión do virial foi proposta por primeira vez por __Kamerlingh Onnes__, un físico que gañou o premio __Nobel__ en 1911 polo seu traballo sobre superconductividade e o helio líquido. Tamé acuñou a palabre _enthalpía_).

A idea central era crear unha serie infinita de aproximación para o factor de compresibilidade:

\begin{equation}
\frac{P\hat{V}}{RT} = A + \frac{B}{\hat{V}} + \frac{C}{\hat{V}^2} + \frac{D}{\hat{V}^3}\cdots
\end{equation}

onde $A$, $B$, $C$ son os valores dependentes da temperatura coñecidos como o primeiro, segundo, e terceiro coeficintes do virial respectivamente. O caso $A=1$ e $B=C=D=\cdots = 0$ corresponde ó gas ideal. 

Unhe versión usada habitualmente desta serie de expansión supón $A=1$, $B(T)$, e $C=D=\cdots = 0$, ca conseguinte aproximación de $\hat{V} = \frac{RT}{P}$. Entón:

\begin{equation}
\frac{P\hat{V}}{RT} = 1 + \frac{BP}{RT}
\end{equation}

O que se pode simplificar para dar:

\begin{equation}
P= \frac{RT}{\hat{V}-B}
\end{equation}

O velor dependente da temperatura $B$ estímase por:

\begin{align*}
B_0 & = 0.083 - \frac{0.422}{T_r^{1.6}} \\
B_1 & = 0.139 - \frac{0.172}{T_r^{4.2}} \\
B & = \frac{RT_c}{P_c}\left(B_0 + \omega B_1\right)
\end{align*}

$\omega$ é o **factor acéntrico de Pitzer**, cuxos valores están en varias táboas de datos químicos de sustancias.

In [5]:
temperatura_reducida = temperatura / temperatura_critica
B0 = 0.083 - 0.422 / temperatura_reducida ** 1.6
B1 = 0.139 - 0.172 / temperatura_reducida ** 4.2
B = (R * temperatura_critica / presion_critica) * (B0 + factor_acentrico * B1)
print('B =', B)

presion_virial = R * temperatura / ( volume_especifico - B)

resultados['Virial'] = presion_virial

print('presión de CO2 =', round(presion_virial, 2), 'atm', round(14.696*presion_virial, 2), 'psia')
print('presión de CO2 calculada polo virial = {:.2f} atm ou {:.2f} psia'.format(presion_virial,14.696 * presion_virial))

B = -0.1105622768302878
presión de CO2 = 146.75 atm 2156.63 psia
presión de CO2 calculada polo virial = 146.75 atm ou 2156.63 psia


## Ecuación de Van der Waals

A ecuación de estado de __Van der Waals__ ven dad por:

$$
\left ( p + \frac {a} {V^2} \right ) \left ( V - b \right ) = RT
$$

onde

$$
a = \frac {27}{64} \left ( \frac {R^2 T^2_C} {p_c} \right )
$$

$$
b + \frac {R T_C} {8 p_C}
$$

A presión reducida defínese como:

$$
p_r = \frac {p} {p_C}
$$

e o factor de compresibilidade, $z$, ven dado por:

$$
z = \frac {pV} {RT}
$$

In [6]:
a = ( (27 / 64 ) * R ** 2 * temperatura_critica ** 2 ) / presion_critica
b = ( R * temperatura_critica ) / ( 8 * presion_critica)

In [7]:
presion_vanderwaals = R * temperatura / ( volume_especifico - b ) - a / volume_especifico ** 2

resultados['Van der Waals'] = presion_vanderwaals

print('Presión de CO2 por Van der Waals =', presion_vanderwaals, 'atm', 14.696 * presion_vanderwaals,'psia')
print('Presión de CO2 por Van der Waals = {:.2f} atm ou {:.2f} psia'.format(presion_vanderwaals, 14.696 * presion_vanderwaals,'psia'))

Presión de CO2 por Van der Waals = 317.43819738541583 atm 4665.071748776071 psia
Presión de CO2 por Van der Waals = 317.44 atm ou 4665.07 psia


## Ecuación de Estado de Soave-Redlich-Kwong

A EOS de Soave-Redlick-Kwong é unha das ecuacións de estado mais usadas porque é aplicable a unha ampla variedad de sistemas. A expresión xeral da ecuación é:

\begin{equation}
P = \frac{RT}{\hat{V}-b} - \frac{\alpha a}{\hat{V}(\hat{V}+b)}
\end{equation}

onde os parámetros da mesma veñen dados por:

\begin{align*}
a & = 0.42747 \frac{(RT_c)^2}{P_c} \\
b & = 0.08664 \frac{RT_c}{P_c} \\
m & = 0.48508 + 1.55171\omega - 0.1561\omega^2\\
T_r & = \frac{T}{T_c}\\
\alpha & = \left[1 + m(1 - \sqrt{T_r})\right]^2
\end{align*}

In [8]:
from math import sqrt

a = 0.42747 * ( R * temperatura_critica) ** 2 / presion_critica
b = 0.08664 * R * temperatura_critica / presion_critica
m = 0.48508 + 1.5517 * factor_acentrico - 0.1561 * factor_acentrico ** 2
temperatura_reducida = temperatura / temperatura_critica
alpha = ( 1 + m * ( 1 - sqrt ( temperatura_reducida ) ) ) **2

presion_srk = R * temperatura / ( volume_especifico - b ) - alpha * a / volume_especifico / ( volume_especifico + b )

resultados['SRK'] = presion_srk

print('Presión de CO2 por Soave-Redlich-Kwong =', presion_srk, 'atm', 14.696 * presion_srk, 'psia')
print('Presión de CO2 por Soave-Redlich-Kwong = {:.2f} atm ou {:.2f} psia'.format(presion_srk, 14.696 * presion_srk) )

Presión de CO2 por Soave-Redlich-Kwong = 150.71148961601057 atm 2214.8560513968914 psia
Presión de CO2 por Soave-Redlich-Kwong = 150.71 atm ou 2214.86 psia


### Datos de referencia do NIST Webbook

O NIST (National Institute of Standards and Technology) mantén unha base de datos accesible por web que contén datos de referencia. O [Webbook do NIST](https://webbook.nist.gov/chemistry/) é un sitio moi fiable donde se poden atopar datos fiables e constrastados de mais de 7000 compostos químicos orgáncios e inorgánicos.

### [Datos de Propiedades do fluído do NIST Fluid para o $CO_2$](https://webbook.nist.gov/cgi/fluid.cgi?ID=C124389&Action=Page)


In [9]:
densidade = 1 / volume_especifico
print('Densidade = {:.2f} mol/L'.format(densidade))
# puntos a interpolar
pb = 118.43 
db = 16.309
pa = 105.27 
da = 15.139

presion_nist = pa + ( densidade - da ) * ( pb - pa ) /  ( db - da )
print(presion_nist)

resultados['NIST Webbook'] = presion_nist

print('Presión CO2 NIST Webbok =', presion_nist, 'atm', 14.696 * presion_nist, 'psi')
print('Presión CO2 NIST Webbok = {:.3f} atm ou {:.3f} psi'.format(presion_nist,14.696 * presion_nist))

Densidade = 15.49 mol/L
109.24835581741479
Presión CO2 NIST Webbok = 109.24835581741479 atm 1605.5138370927277 psi
Presión CO2 NIST Webbok = 109.248 atm ou 1605.514 psi


In [10]:
print('{0:15s}  {1:9s}   {2:5s}'.format('EOS', 'Presión', 'Error'))

for key,val in resultados.items():
    err = 100 * ( ( val - presion_nist ) / presion_nist )
    print('{0:18s}  {1:5.1f} atm   {2:5.1f}%'.format(key, val, err))
  

EOS              Presión     Error
Gas ideal           398.1 atm   264.4%
Compresibilidade    111.5 atm     2.0%
Virial              146.7 atm    34.3%
Van der Waals       317.4 atm   190.6%
SRK                 150.7 atm    38.0%
NIST Webbook        109.2 atm     0.0%
